<a href="https://colab.research.google.com/github/sthalles/SimCLR/blob/master/feature_eval/mini_batch_logistic_regression_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import sys
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier
import yaml
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import importlib.util

In [2]:
def get_file_id_by_model(folder_name):
  file_id = {'resnet-18_40-epochs': '1c4eVon0sUd-ChVhH6XMpF6nCngNJsAPk',
             'resnet-18_80-epochs': '1L0yoeY9i2mzDcj69P4slTWb-cfr3PyoT',
             'resnet-50_40-epochs': '1TZqBNTFCsO-mxAiR-zJeyupY-J2gA27Q',
             'resnet-50_80-epochs': '1is1wkBRccHdhSKQnPUTQoaFkVNSaCb35',
             'resnet-18_100-epochs':'1aZ12TITXnajZ6QWmS_SDm8Sp8gXNbeCQ'}
  return file_id.get(folder_name, "Model not found.")

In [3]:
folder_name = '/home/radhakrishnan.39a/Documents/NN/SimCLR-2/runs/Apr25_15-32-48_cse-dcse101551d.coeit.osu.edu/'
file_id = get_file_id_by_model(folder_name)
print(folder_name, file_id)

/home/radhakrishnan.39a/Documents/NN/SimCLR-2/runs/Apr25_15-32-48_cse-dcse101551d.coeit.osu.edu/ Model not found.


In [4]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets

In [5]:
device = torch.device('cuda:0')
print("Using device:", device)

Using device: cuda:0


In [6]:
checkpoints_folder = os.path.join(folder_name, 'checkpoints')
config = yaml.load(open(os.path.join(checkpoints_folder, "config.yaml"), "r"))
config

<ipython-input-6-42da58e34c2b>:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(open(os.path.join(checkpoints_folder, "config.yaml"), "r"))


{'batch_size': 512,
 'epochs': 80,
 'eval_every_n_epochs': 1,
 'fine_tune_from': 'None',
 'log_every_n_steps': 50,
 'weight_decay': '10e-6',
 'fp16_precision': False,
 'model': {'out_dim': 256, 'base_model': 'resnet18'},
 'dataset': {'data_path': '/home/radhakrishnan.39a/Documents/NN/Sentinel_Dataset',
  's': 1,
  'input_shape': '(64,64,3)',
  'num_workers': 0,
  'valid_size': 0.05},
 'loss': {'temperature': 0.5, 'use_cosine_similarity': True}}

In [7]:
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.datasets import ImageFolder
import math

In [16]:
train_dataset = ImageFolder('/home/radhakrishnan.39a/Documents/NN/Sentinel_Dataset/train_10000/',transform=transforms.ToTensor())

test_dataset = ImageFolder('/home/radhakrishnan.39a/Documents/NN/Sentinel_Dataset/test/',transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=100,
                            num_workers=0, drop_last=False, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=500,
                            num_workers=0, drop_last=False, shuffle=False)

In [9]:
def _load_resnet_model(checkpoints_folder):
  # Load the neural net module
  spec = importlib.util.spec_from_file_location("model", os.path.join(checkpoints_folder, 'resnet_simclr.py'))
  resnet_module = importlib.util.module_from_spec(spec)
  spec.loader.exec_module(resnet_module)

  model = resnet_module.ResNetSimCLR(**config['model'])
  model.eval()

  state_dict = torch.load(os.path.join(checkpoints_folder, 'model.pth'), map_location=torch.device('cpu'))
  model.load_state_dict(state_dict)
  model = model.to(device)
  return model

## Logisitc Regression LinearEval

In [10]:
class ResNetFeatureExtractor(object):
  def __init__(self, checkpoints_folder, train_loader, test_loader):
    self.checkpoints_folder = checkpoints_folder
    self.model = _load_resnet_model(checkpoints_folder)
    self.train_loader = train_loader
    self.test_loader = test_loader

  def _inference(self, loader):
    feature_vector = []
    labels_vector = []
    for batch_x, batch_y in loader:

      batch_x = batch_x.to(device)
      labels_vector.extend(batch_y)

      features, _ = self.model(batch_x)
      feature_vector.extend(features.cpu().detach().numpy())

    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)

    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector

  def get_resnet_features(self):
    train_loader, test_loader = self.train_loader, self.test_loader
    X_train_feature, y_train = self._inference(train_loader)
    X_test_feature, y_test = self._inference(test_loader)

    return X_train_feature, y_train, X_test_feature, y_test

In [17]:
resnet_feature_extractor = ResNetFeatureExtractor('/home/radhakrishnan.39a/Documents/NN/SimCLR-2/runs/Apr25_15-32-48_cse-dcse101551d.coeit.osu.edu/checkpoints/', train_loader, test_loader)

Feature extractor: resnet18


In [18]:
X_train_feature, y_train, X_test_feature, y_test = resnet_feature_extractor.get_resnet_features()

Features shape (10000, 512)
Features shape (15240, 512)


In [13]:
import torch.nn as nn

class LogisticRegression(nn.Module):
    
    def __init__(self, n_features, n_classes):
        super(LogisticRegression, self).__init__()
        self.model = nn.Linear(n_features, n_classes)

    def forward(self, x):
        return self.model(x)

In [19]:
class LogiticRegressionEvaluator(object):
  def __init__(self, n_features, n_classes):
    self.log_regression = LogisticRegression(n_features, n_classes).to(device)
    self.scaler = preprocessing.StandardScaler()

  def _normalize_dataset(self, X_train, X_test):
    print("Standard Scaling Normalizer")
    self.scaler.fit(X_train)
    X_train = self.scaler.transform(X_train)
    X_test = self.scaler.transform(X_test)
    return X_train, X_test

  @staticmethod
  def _sample_weight_decay():
    # We selected the l2 regularization parameter from a range of 45 logarithmically spaced values between 10−6 and 105
    weight_decay = np.logspace(-6, 5, num=45, base=10.0)
    weight_decay = np.random.choice(weight_decay)
    print("Sampled weight decay:", weight_decay)
    return weight_decay

  def eval(self, test_loader):
    correct = 0
    total = 0
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    with torch.no_grad():
      self.log_regression.eval()
      for batch_x, batch_y in test_loader:
          batch_x, batch_y = batch_x.to(device), batch_y.to(device)
          logits = self.log_regression(batch_x)

          prediction = torch.argmax(logits, dim=1)
          total += batch_y.size(0)
          correct += (prediction == batch_y).sum().item()
          for i in range(len(prediction.int())):
#             if (prediction.int()[i] == labels.data.int()[i]):
#                 acc_c+=1
            if ((prediction[i] == 0) and (batch_y.data[i] == 0)):
                tp +=1
            elif ((prediction[i] == 0) and (batch_y.data[i] == 1)):
                fp+=1
                #fpl.append(fnames[i])
            elif ((prediction[i] == 1) and (batch_y.data[i] == 1)):
                tn+=1
                #tnl.append(fnames[i])
            elif ((prediction[i] == 1) and (batch_y.data[i] == 0)):
                fn+=1
                #fnl.append(fnames[i])
        
        

      final_acc = 100 * correct / total
      prec = tp/(tp+fp)
      rec = tp/(tp+fn)
      f1 = 2 * ((prec*rec)/(prec+rec))
    
      self.log_regression.train()
      return final_acc, prec, rec, f1

  def checkpoint_eval(self, ckp_fp, X_test, y_test):
    correct = 0
    total = 0
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    state_dict = torch.load(ckp_fp)
    test = torch.utils.data.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test).type(torch.long))
    test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
    self.log_regression.load_state_dict(state_dict)
    with torch.no_grad():
      self.log_regression.eval()
      for batch_x, batch_y in test_loader:
          batch_x, batch_y = batch_x.to(device), batch_y.to(device)
          logits = self.log_regression(batch_x)

          prediction = torch.argmax(logits, dim=1)
          total += batch_y.size(0)
          correct += (prediction == batch_y).sum().item()
          for i in range(len(prediction.int())):
#             if (prediction.int()[i] == labels.data.int()[i]):
#                 acc_c+=1
            if ((prediction[i] == 0) and (batch_y.data[i] == 0)):
                tp +=1
            elif ((prediction[i] == 0) and (batch_y.data[i] == 1)):
                fp+=1
                #fpl.append(fnames[i])
            elif ((prediction[i] == 1) and (batch_y.data[i] == 1)):
                tn+=1
                #tnl.append(fnames[i])
            elif ((prediction[i] == 1) and (batch_y.data[i] == 0)):
                fn+=1
                #fnl.append(fnames[i])
        
        

      final_acc = 100 * correct / total
      prec = tp/(tp+fp)
      rec = tp/(tp+fn)
      f1 = 2 * ((prec*rec)/(prec+rec))
    
      self.log_regression.train()
      print(f"Accuracy: {final_acc}, precision: {prec}, recall: {rec}, F1: {f1}")
      return final_acc, prec, rec, f1


  def create_data_loaders_from_arrays(self, X_train, y_train, X_test, y_test):
    X_train, X_test = self._normalize_dataset(X_train, X_test)

    train = torch.utils.data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train).type(torch.long))
    train_loader = torch.utils.data.DataLoader(train, batch_size=396, shuffle=False)

    test = torch.utils.data.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test).type(torch.long))
    test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
    return train_loader, test_loader

  def train(self, X_train, y_train, X_test, y_test):
    
    train_loader, test_loader = self.create_data_loaders_from_arrays(X_train, y_train, X_test, y_test)

    weight_decay = self._sample_weight_decay()

    optimizer = torch.optim.Adam(self.log_regression.parameters(), 3e-4, weight_decay=weight_decay)
    criterion = torch.nn.CrossEntropyLoss()

    best_accuracy = 0

    for e in range(200):
      
      for batch_x, batch_y in train_loader:

        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()

        logits = self.log_regression(batch_x)

        loss = criterion(logits, batch_y)

        loss.backward()
        optimizer.step()

      epoch_acc, prec, rec, f1 = self.eval(test_loader)
      
      if epoch_acc > best_accuracy:
        #print("Saving new model with accuracy {}".format(epoch_acc))
        best_accuracy = epoch_acc
        best_prec = prec
        best_rec = rec
        best_f1 = f1
        torch.save(self.log_regression.state_dict(), 'log_regression.pth')

    print("--------------")
    print("Done training")
    print(f"Best accuracy: {best_accuracy}, precision: {best_prec}, recall: {best_rec}, F1: {best_f1}")

In [19]:
#Train with 1000 labeled examples
log_regressor_evaluator = LogiticRegressionEvaluator(n_features=X_train_feature.shape[1], n_classes=2)

log_regressor_evaluator.train(X_train_feature, y_train, X_test_feature, y_test)

Standard Scaling Normalizer
Sampled weight decay: 1e-06
--------------
Done training
Best accuracy: 92.44094488188976, precision: 0.9248554913294798, recall: 0.9238845144356955, F1: 0.9243697478991597


In [15]:
#Train with 5000 labeled examples
log_regressor_evaluator = LogiticRegressionEvaluator(n_features=X_train_feature.shape[1], n_classes=2)

log_regressor_evaluator.train(X_train_feature, y_train, X_test_feature, y_test)

Standard Scaling Normalizer
Sampled weight decay: 0.1778279410038923
--------------
Done training
Best accuracy: 92.59186351706036, precision: 0.9185041908446164, recall: 0.9347769028871391, F1: 0.9265691056910569


In [20]:
#Train with 10000 labeled examples
log_regressor_evaluator = LogiticRegressionEvaluator(n_features=X_train_feature.shape[1], n_classes=2)

log_regressor_evaluator.train(X_train_feature, y_train, X_test_feature, y_test)

Standard Scaling Normalizer
Sampled weight decay: 100.0
--------------
Done training
Best accuracy: 89.16666666666667, precision: 0.8660615724273274, recall: 0.9266404199475066, F1: 0.8953274583148418


In [31]:
#Train with 15k labeled examples
log_regressor_evaluator = LogiticRegressionEvaluator(n_features=X_train_feature.shape[1], n_classes=2)
log_regressor_evaluator.checkpoint_eval('./log_regression.pth', X_test_feature, y_test)

Accuracy: 88.04461942257218, precision: 0.8877742108079186, recall: 0.870997375328084, F1: 0.8793057763645998


(88.04461942257218, 0.8877742108079186, 0.870997375328084, 0.8793057763645998)